### Recommendation System - K arm Bandit Problem

In [1]:
import numpy as np
from abc import ABC, abstractmethod
from sklearn.tree import DecisionTreeClassifier  
from sklearn.base import clone                  
import matplotlib.pyplot as plt                
from sklearn.metrics.pairwise import rbf_kernel   

In [2]:
class MAB(ABC):
    def __init__(self, n_arms): # n_arms: number of arms
        self.n_arms = n_arms
        
    @abstractmethod
    def play(self, context): 
        self.context = context

    
    @abstractmethod
    def update(self, arm, reward, context): 
        self.arm = arm
        self.reward = reward
        self.context = context


In [3]:
def break_tie(_range):
    indices = np.argwhere(_range == np.max(_range))
    index = np.random.randint(0,len(indices))
    return indices[index][0]

`Dataset`
- Column 1: The arm played by a uniformly-random policy out of 10 arms (news articles)
- Column 2: The reward received from the arm played (1 if the user clicked 0 otherwise)
- Columns 3-102: The 100-dim flattened context; 10 features per arm (incorporating the content of the article and its match with the visiting user), first the features for arm 1, then arm 2, etc. up to arm 10.

In [4]:
data = np.loadtxt("dataset.txt")
print(data.shape)
print(data[0])

(10000, 102)
[ 1.  0.  5.  0.  0. 37.  6.  0.  0.  0.  0. 25.  0.  0.  7.  1.  0.  0.
  0. 13.  2.  0.  0.  8.  0.  0.  0. 15. 29.  1.  1.  0.  0.  0.  0.  1.
  0.  0.  1.  0.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  4.  0.  0.  0.  0.  0.  0.  0.  5.  2.  7.  0.  0.  0.  0. 11.  0.
  5.  0.  0.  0.  0.  0.  0.  3.  2.  0.  0.  0.  0.  1.  2. 47.  0.  0.
  1.  0.  0.  0.  1.  3.  0.  0. 17. 30.  4.  0.]


In [5]:
arms, rewards, contexts = data[:,0], data[:,1], data[:,2:] # arm: 1st column, reward: 2nd column, context: 3rd to last column
print(type(arms), type(rewards), type(contexts))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [6]:
arms = arms.astype(int)
rewards = rewards.astype(float)
contexts = contexts.astype(float)

In [7]:
arms[0], rewards[0], contexts[0]

(1,
 0.0,
 array([ 5.,  0.,  0., 37.,  6.,  0.,  0.,  0.,  0., 25.,  0.,  0.,  7.,
         1.,  0.,  0.,  0., 13.,  2.,  0.,  0.,  8.,  0.,  0.,  0., 15.,
        29.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  2.,  7.,  0.,
         0.,  0.,  0., 11.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,
         2.,  0.,  0.,  0.,  0.,  1.,  2., 47.,  0.,  0.,  1.,  0.,  0.,
         0.,  1.,  3.,  0.,  0., 17., 30.,  4.,  0.]))

In [8]:
print('Unique arms:', np.unique(arms))
print('Unique rewards:', np.unique(rewards))

Unique arms: [0 1 2 3 4 5 6 7 8 9]
Unique rewards: [0. 1.]


In [9]:
n_arms = len(np.unique(arms))
n_events = len(contexts)
n_dims = int(len(contexts[0])/n_arms)

contexts = contexts.reshape(n_events, n_arms, n_dims)
print(contexts.shape)

(10000, 10, 10)


In [10]:
def offlineEvaluate(mab, arms, rewards, contexts, n_rounds=None):
    n_round = 0
    R = []          
    H = []          
    
    for i in range(n_events):
        if n_round == n_rounds:
            break
        arm = mab.play(contexts[i])
        if arm == arms[i]:                 
            R.append(rewards[i])           
            H.append([arms[i], rewards[i], contexts[i]])     
            mab.update(arms[i], rewards[i], contexts[i])     
            n_round += 1

    out = np.array(R)
        
    return out 

In [11]:
class EpsGreedy(MAB):
    def __init__(self, n_arms, epsilon, Q0=np.inf):
        super().__init__(n_arms)
        self.epsilon = epsilon
        self.q = np.full(n_arms, Q0)
        self.rewards = np.zeros(n_arms)
        self.clicks = np.zeros(n_arms)
    
    def play(self, context=None):
        super().play(context)
        if np.random.random_sample() <= self.epsilon:
            arm = np.random.randint(0, self.n_arms)
        else:
            arm = break_tie(self.q)
        return arm
    
    def update(self, arm, reward, context=None):
        super().update(arm, reward, context)
        self.clicks[arm] += 1
        self.rewards[arm] += self.reward
        self.q[arm] = self.rewards[arm] / self.clicks[arm]

In [12]:
mab = EpsGreedy(10, 0.05) 
print(mab.play(contexts[0]))

8


In [13]:
print(type(arms), arms.shape)
print(n_events)

<class 'numpy.ndarray'> (10000,)
10000


In [14]:
results_EpsGreedy = offlineEvaluate(mab, arms, rewards, contexts, 800)
print('EpsGreedy average reward', np.mean(results_EpsGreedy))

EpsGreedy average reward 0.26625
